In [1]:
import gensim
import pymystem3

In [2]:
# Функция возвращает список стихов с названиями
def parse(text):
    verse = ''
    verses = []
    for line in text:
        if line.startswith('МЕТКА'):
            continue
        elif line.startswith('НАЗВАНИЕ'):
            title = line.replace('НАЗВАНИЕ', '').strip()
            if not title.isnumeric():
                verse += title + '\n\n'
        elif line == 'ТЕКСТ\n':
            continue
        elif line == 'КОН\n':
            verses.append(verse)
            verse = ''
        else:
            verse += line
    
    return verses

In [3]:
# Функция преобразует список стихов в список списков токенов вида лемма_ЧАСТЬРЕЧИ
# Классификация частей речи в формате Mystem
# Возможен учёт стоп-слов
def preprocess(texts, stop_words=None):
    mystem = pymystem3.Mystem()
    preprocessed_texts = []
    
    for text in texts:
        preprocessed_text = []
        analized = mystem.analyze(text)
        
        for result in analized:
            if 'analysis' not in result:
                continue
            if result['analysis'] == []:
                continue
            lemma = result['analysis'][0]['lex']
            if stop_words is not None:
                if lemma in stop_words:
                    continue
            pos = result['analysis'][0]['gr'].split(',')[0].split('=')[0]
            preprocessed_text.append(lemma.lower() + '_' + pos)
            
        preprocessed_texts.append(preprocessed_text)
        
    return preprocessed_texts

In [4]:
# Создаёт и возвращает обученную модель векторизованных токенов
def create_model(preprocessed_texts, size=150, window=15, min_count=1, epochs=15):
    model = gensim.models.Word2Vec(preprocessed_texts,
                                   size=size, window=window, min_count=min_count)
    model.train(preprocessed_texts, total_examples=len(preprocessed_texts),
                epochs=epochs)
    return model

In [5]:
# Возвращает список текстов из texts, которые удовлетворяют запросу по слову word
# topn - количество "похожих" слов, также используемых при поиске
# log - показать список слов, использвуемых при поиске (слово + похожие)
def search(word, model, texts, preprocessed_texts, topn=10, log=False):
    mystem = pymystem3.Mystem()
    analyzed = mystem.analyze(word)
    try:
        if 'analysis' not in analyzed[0]:
            raise KeyError('Incorrect search queue')
        if analyzed[0]['analysis'] == []:
            raise KeyError('Incorrect word')
        lemma = analyzed[0]['analysis'][0]['lex']
        pos = analyzed[0]['analysis'][0]['gr'].split(',')[0].split('=')[0]
        word = lemma + '_' + pos
        if log:
            print('Ищем: ' + word + ' ...')

        searched_texts = set()
        for i in range(len(preprocessed_texts)):
            if word in preprocessed_texts[i]:
                searched_texts.add(texts[i])
        if topn > 0:
            if log:
                print('А также:')
            for result in model.wv.most_similar(word, topn=topn):
                similar_word = result[0]
                if log:
                    print(similar_word)
                for i in range(len(preprocessed_texts)):
                    if similar_word in preprocessed_texts[i]:
                        searched_texts.add(texts[i])
            if log:
                print()
        return list(searched_texts)
    except KeyError:
        return ['Поиск не дал результатов']

In [6]:
# При препроцессировании можно учесть стоп-слова, хотя заумь лучше оставить как есть
f = open('stop_words.txt', 'r')
stop_words = []
for line in f:
    stop_words.append(line.strip())

In [7]:
# Обучим модель на сборнике Миши, не беря последние 3 стихотворения
verses = parse(open('taskO1_sokolov.txt', 'r').readlines())[:-3]
preprocessed_verses = preprocess(verses)
model = create_model(preprocessed_verses)

In [8]:
# Как выглядит словарь нашей модели
list(model.wv.vocab)[:10]

['что_SPRO',
 'это_PART',
 'так_ADVPRO',
 'красный_A',
 'рот_S',
 'у_PR',
 'жаба_S',
 'не_PART',
 'жевать_V',
 'ль_PART']

In [9]:
# Попробуем поискать знаменитое слово
# Пока что поиск может искать только слова!
for result in search('ко', model, verses, preprocessed_verses, topn=6, log=True):
    print(result + '\n')

Ищем: ко_PR ...
А также:
романюк_S
анечка_S
колючий_A
озарь_S
гад_S
жека_S

ОН

кобзарь
озарь
грандиозарь
виртуозарь


No one cares

КОМЕТА ЗАБИЛАСЬ ко мне ПОД ПОДУШКУ
Жужжит и щекочет, целуя колючее ушко


No one cares

Кокетничая запонками
из свеже-отравленных скорпионов
Портовый кран
вдвое вытянул
изумрудный перископ головы
и прикрыл
индиговым сатином
жабры,
дразня пролетающих с Олимпа
алебастровых богинь
цин-ко-но-жек!..


No one cares

СМЕРТЬ ХУДОЖНИКА
привыкнув ко всем безобразьям
искал я их днем с фонарем
но увы! все износились проказы
не забыться мне ни на чем!
и взор устремивши к бесплотным
я тихо но твердо сказал:
мир вовсе не рвотное —
и мордой уткнулся в Обводный канал…


Анечке Романюк

ко ко
ко ко ко
ко ко
ко ко ко
ко ко
ко ко ко
ко
ко
ко ко
ко ко ко
ко
ко ко ко
ко
ко ко ко
ко
ко ко ко
ко ко
ко ко
ко ко
ко ко


ОЛЕСЯ

Я села, Олеся,
мо... да с адом
тет-а-тет.
Цена: танец
тенет.
Я... но могу, угомоня
ок темь метко...
О, лежать тяжело.
О, дух, худо
ей, ей-ей.
Там, там — мат

In [10]:
# Попробуем поискать слово, которое содержится в одном из последних трёх стихотворений
for result in search('слизняк', model, verses, preprocessed_verses, log=True):
    print(result + '\n')

Ищем: слизняк_S ...
А также:
Поиск не дал результатов



In [11]:
# Дообучает модель новыми списками токенов
def update_model(model, preprocessed_texts, epochs=15):
    model.build_vocab(preprocessed_texts, update=True)
    model.train(preprocessed_texts, total_examples=len(preprocessed_texts),
                epochs=epochs)

In [17]:
# Дообучим нашу модель
new_verses = parse(open('taskO1_sokolov.txt', 'r').readlines())[-3:]
preprocessed_new_verses = preprocess(new_verses)
print(preprocessed_new_verses[0])
update_model(model, preprocessed_new_verses)

['дым_S', 'накрасить_V', 'ноздря_S', 'курчавоглазый_A', 'зверек_S', 'толчок_S', 'сдуть_V', 'я_SPRO', 'с_PR', 'площадка_S', 'воздух_S', 'и_CONJ', 'я_SPRO', 'лететь_V', 'как_CONJ', 'выронить_V', 'слизняк_S']


In [13]:
# Проверим словарь
list(model.wv.vocab)[:10]

['что_SPRO',
 'это_PART',
 'так_ADVPRO',
 'красный_A',
 'рот_S',
 'у_PR',
 'жаба_S',
 'не_PART',
 'жевать_V',
 'ль_PART']

In [14]:
# Проверим, что новое слово теперь есть в словаре
'слизняк_S' in model.wv.vocab

True

In [15]:
# Теперь он должен уметь искать новое слово и сам текст, в котором оно содержится
for result in search('слизняк', model, verses + new_verses,
                     preprocessed_verses + preprocessed_new_verses, topn=1, log=True):
    print(result + '\n')

Ищем: слизняк_S ...
А также:
как_CONJ

No one cares

Чисто по женски нежно и ласково
Она убеждает, что я талант
Что меня по меню положат на — стол
И будут все как лучший ужин захлебываясь лакать
Ватага изысканных жевак
Набросится на мою телячью ножку
Кину им пачку улыбок золотых рыбок
Будут пораженные плясать до утра бряцая воистину ложками
Запивая ликером моей цветущей рубахи,
Где на подтяжках висит красного дерева диван
И стану в угол и буду от восхищение
и благодарности плакать
а за мною
Весь кафе-ресторан…


шопышин А шопышин А шопышин А шопышин а
ноет поперечен крыло поломали стало как в бочке стало сельдь
была голова круглой хвост был длинный глаза два гривенника агарусный ШАРФ
ты отчего ржавая
отчего на простынях
отчего при лампочке
УшЕЛОГая
Вспыхнул керосин и потух полосы ночные пошли и выглянув из-за лампы пуп и застыл
Плесневая струйка ползет в ней царствует бензИН в узком платьЕ
Струйка тир-тир-тир лиски это не отверстие мячик гриб в углу
а над грибом одна звездочка и страшн